In [1]:
library(rnoaa)
library(tidyverse)
library(lubridate)
library(rlang)
library(stringr)

Warning message:
"package 'rnoaa' was built under R version 4.1.2"
Registered S3 method overwritten by 'httr':
  method           from  
  print.cache_info hoardr

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.0.2     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 4.1.2"
Warning message:
"package 'tidyr' was built under R version 4.1.2"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union



Attaching package: 'rlang'


The following objects are masked from 'package:purrr':

    %@%, as_function, 

In [2]:
stations <- ghcnd_stations()

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/ghcnd-stations.rds

date created (size, mb): 2022-02-12 20:25:20 (2.159)

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/ghcnd-inventory.rds

date created (size, mb): 2022-02-12 20:26:28 (2.669)



In [4]:
# load df. don't change df names as they will be used as input of function

kyoto <- read.csv("raw/kyoto.csv")
liestal <- read.csv("raw/liestal.csv")
washingtondc <- read.csv("raw/washingtondc.csv")

kyoto<- kyoto|> 
  mutate(year = strtoi(substr(year, 1, 4)))

japan <- read.csv("raw/japan.csv")
swiss <- read.csv("raw/meteoswiss.csv")
south_korea <- read.csv("raw/south_korea.csv")

In [5]:
# filtering the city data to roughly match station id geographically
# using 1 decimal as approximation

stations <- stations |> 
  mutate(latr = round(latitude, 1), longr = round(longitude, 1))

# need to update to follow DRY

japan <- japan |> 
  mutate(latr = round(lat, 1), longr = round(long, 1))
swiss <- swiss |> 
  mutate(latr = round(lat, 1), longr = round(long, 1))
south_korea <- south_korea |> 
  mutate(latr = round(lat, 1), longr = round(long, 1))


In [6]:
# retrieve approximate station id and row bind them

japan_id <- merge(japan, stations, by = c("latr", "longr"))
swiss_id <- merge(swiss, stations, by = c("latr", "longr"))
south_korea <- merge(south_korea, stations, by = c("latr", "longr"))
all <- rbind(japan_id, swiss_id, south_korea)

In [7]:
# combine every location together

all <- all |> 
  select(id, location, lat, long, alt, year, bloom_date, bloom_doy)

In [8]:
# add id to three prediction cities

kyoto <- kyoto |> 
  mutate(id = "JA000047759")
washingtondc <- washingtondc |> 
  mutate(id = "USC00186350")
liestal <- liestal |> 
  mutate(id = "GME00127786")

all_pred <- rbind(kyoto, washingtondc, liestal) |> 
  select(id, location, lat, long, alt, year, bloom_date, bloom_doy)

In [9]:
all <- rbind(all, all_pred) |> 
    distinct()

In [10]:
# retrieve unique station ids for all location and para list

id_list <- unique(all$id)
para_list <- c("tmax", "tmin", "prcp", "swnd")

In [14]:
# left join climate data to cherry data

# result <- all

# for (para in para_list){
#     for (id in id_list){
#         df <- ghcnd_search(stationid = id, date_min = "1950-01-01", date_max = "2022-01-31")[[para]]
#         if (is.null(df) == FALSE){
#             tr <- df |> 
#                 mutate(year = as.integer(year(date)),
#                     !!sym(para) := !!sym(para) / 10) |>
#                 drop_na() |>
#                 select(id, !!sym(para), year) |> 
#                 group_by(year, id) |> 
#                 summarise(!!sym(para) := mean(!!sym(para), na.rm = TRUE))
#             result <- left_join(result, tr, by = c("year", "id"))
            
#             para1 <- paste(para, ".x", sep = "")
#             para2 <- paste(para, ".y", sep = "")
#             if (para1 %in% names(result)){
#                 result <- result |> 
#                     mutate(!!sym(para) := coalesce(!!sym(para1), !!sym(para2))) |> 
#                     select(-!!sym(para1), -!!sym(para2))
#             }

#         }

#     }
# }

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047918.dly

date created (size, mb): 2022-02-21 23:37:19 (1.187)

file min/max dates: 1973-01-01 / 2021-10-31

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JA000047912.dly

date created (size, mb): 2022-02-22 00:04:24 (0.686)

file min/max dates: 1957-01-01 / 1989-12-31

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047927.dly

date created (size, mb): 2022-02-22 00:04:50 (1.185)

file min/max dates: 1973-01-01 / 2021-10-31

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAW00042206.dly

date created (size, mb): 2022-02-22 00:04:54 (0.973)

file min/max dates: 1949-05-01 / 1968

In [12]:
# result <- result|> 
    # drop_na()

In [13]:
# write.csv(result,"processed/clean.csv", row.names = FALSE)

In [ ]:
# for (para in para_list){
#     for (id in id_list){
#         df <- ghcnd_search(stationid = id, date_min = "1950-01-01", date_max = "2022-01-31")[[para]]
#         if (is.null(df) == FALSE){
#             tr <- df |> 
#                 mutate(year = as.integer(year(date)),
#                     !!sym(para) := !!sym(para) / 10) |>
#                 drop_na() |>
#                 select(id, !!sym(para), year) |> 
#                 group_by(year, id) |> 
#                 summarise(!!sym(para) := mean(!!sym(para), na.rm = TRUE))
#             result <- left_join(result, tr, by = c("year", "id"))
            
#             para1 <- paste(para, ".x", sep = "")
#             para2 <- paste(para, ".y", sep = "")
#             if (para1 %in% names(result)){
#                 result <- result |> 
#                     mutate(!!sym(para) := coalesce(!!sym(para1), !!sym(para2))) |> 
#                     select(-!!sym(para1), -!!sym(para2))
#             }

#         }

#     }
# }

In [61]:
mlist <- c(12, 1, 2)
lagnum <- 1
idlist <- unique(result$id)
tbase <- 0

for (id in idlist){
    d <- ghcnd_search(stationid = id, date_min = "1950-01-01", date_max = "2022-01-31")[["tmax"]]
    f <- ghcnd_search(stationid = id, date_min = "1950-01-01", date_max = "2022-01-31")[["tmin"]]
    df <- merge(d, f, by = c("date", "id")) |> 
      select(date, id, tmax, tmin) |> 
      mutate(year = year(date), month = month(date), tmax = tmax / 10, tmin = tmin / 10) |>
      mutate(gdd = (tmax + tmin) / 2 - tbase) |> 
      drop_na()|> 
      mutate(gdd = case_when(gdd < 0 ~ 0,
                             TRUE ~ gdd)) |> 
      group_by(id, year, month) |> 
      summarise(agdd = sum(gdd, na.rm = TRUE)) |> 
      ungroup() |> 
      mutate(mu = case_when(month %in% mlist ~ "yes",
                            TRUE ~ "no")) |> 
      mutate(lag_agdd = lag(agdd, lagnum),
             lag_mu = lag(mu, lagnum)) |> 
      drop_na() |> 
      filter(lag_mu == "yes") |> 
      group_by(id, year, lag_mu) |>
      summarise(agdd_winter = sum(lag_agdd, na.rm = TRUE)) |> 
      select(-lag_mu)
    
    result <- left_join(result, df, by = c("year", "id"))
    
    if ("agdd_winter.x" %in% names(result)){
        result <- result |> 
            mutate(agdd_winter = coalesce(agdd_winter.x, agdd_winter.y)) |> 
            select(-agdd_winter.x, -agdd_winter.y)
    }
}

result

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047918.dly

date created (size, mb): 2022-02-21 23:37:19 (1.187)

file min/max dates: 1973-01-01 / 2021-10-31

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047918.dly

date created (size, mb): 2022-02-21 23:37:19 (1.187)

file min/max dates: 1973-01-01 / 2021-10-31

`summarise()` has grouped output by 'id', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'id', 'year'. You can override using the `.groups` argument.

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047927.dly

date created (size, mb): 2022-02-22 00:04:50 (1.185)

file min/max dates: 1973-01-01 / 2021-10-31

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047927.dly

date created (size, mb): 2022-02-22 00:04:50 (1.185)

file min/max dates: 1973-01-01 / 2021-10-31

`summarise()` has grouped output by 'id', 'year'. You can override using

id,location,lat,long,alt,year,bloom_date,bloom_doy,tmax,tmin,prcp,agdd_winter
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1973,1973-02-06,37,26.78116,21.81633,6.745170,649.50
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1976,1976-01-28,28,26.51818,20.46606,4.892033,890.00
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1977,1977-02-03,34,27.04046,21.68326,5.897222,875.00
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1978,1978-02-18,49,26.63037,21.04785,5.572299,927.00
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1974,1974-01-20,20,26.76877,21.18828,6.367967,1003.50
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1981,1981-02-02,33,26.41593,21.22707,5.623529,829.00
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1982,1982-01-24,24,26.63745,20.84583,12.083688,733.95
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1983,1983-02-09,40,27.22601,21.71941,10.823881,654.20
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1984,1984-01-26,26,26.72390,21.62684,11.536842,491.15


In [63]:
write.csv(result,"processed/clean.csv", row.names = FALSE)

In [89]:
result <- read.csv("processed/clean.csv")

In [92]:
# generate winter weather parameters

mlist <- c(12, 1, 2)
lagnum <- 1
idlist <- unique(result$id)
# para_list <- c("tmax", "tmin", "prcp", "swnd")

for (id in idlist){
    d <- ghcnd_search(stationid = id, date_min = "1950-01-01", date_max = "2022-01-31")[["tmax"]]
    f <- ghcnd_search(stationid = id, date_min = "1950-01-01", date_max = "2022-01-31")[["tmin"]]
    p <- ghcnd_search(stationid = id, date_min = "1950-01-01", date_max = "2022-01-31")[["prcp"]]
    
        
    df <- merge(d, f, dfby = c("date", "id"))
    df <- merge(df, p, by = c("date", "id"))
    
    df <- df |> 
      select(date, id, tmax, tmin, prcp) |> 
      mutate(year = year(date), month = month(date), tmax = tmax / 10, tmin = tmin / 10) |>
      drop_na()|> 
      group_by(id, year, month) |> 
      summarise(tmax = sum(tmax, na.rm = TRUE),
                tmin = sum(tmin, na.rm = TRUE),
                prcp = sum(prcp, na.rm = TRUE)) |> 
      ungroup() |> 
      mutate(mu = case_when(month %in% mlist ~ "yes",
                            TRUE ~ "no")) |> 
      mutate(lag_tmax = lag(tmax, lagnum),
             lag_tmin = lag(tmin, lagnum),
             lag_prcp = lag(prcp, lagnum),
             lag_mu = lag(mu, lagnum)) |> 
      drop_na() |> 
      filter(lag_mu == "yes") |> 
      group_by(id, year, lag_mu) |>
      summarise(tmax_winter = sum(lag_tmax, na.rm = TRUE),
                tmin_winter = sum(lag_tmin, na.rm = TRUE),
                prcp_winter = sum(lag_prcp, na.rm = TRUE)) |> 
      select(-lag_mu)
    
    result <- left_join(result, df, by = c("year", "id"))
    
    if ("tmax_winter.x" %in% names(result)){
        result <- result |> 
            mutate(tmax_winter = coalesce(tmax_winter.x, tmax_winter.y),
                   tmin_winter = coalesce(tmin_winter.x, tmin_winter.y),
                   prcp_winter = coalesce(prcp_winter.x, prcp_winter.y)) |> 
            select(-tmax_winter.x, -tmax_winter.y,
                   -tmin_winter.x, -tmin_winter.y,
                   -prcp_winter.x, -prcp_winter.y)
    }
}

result

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047918.dly

date created (size, mb): 2022-02-21 23:37:19 (1.187)

file min/max dates: 1973-01-01 / 2021-10-31

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047918.dly

date created (size, mb): 2022-02-21 23:37:19 (1.187)

file min/max dates: 1973-01-01 / 2021-10-31

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047918.dly

date created (size, mb): 2022-02-21 23:37:19 (1.187)

file min/max dates: 1973-01-01 / 2021-10-31

`summarise()` has grouped output by 'id', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'id', 'year'. You can override using the `.groups` argument.

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00047927.dly

date created (size, mb): 2022-02-22 00:04:50 (1.185)

file min/max dates: 1973-01-01 / 2021-10-31

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JAM00

id,location,lat,long,alt,year,bloom_date,bloom_doy,tmax,tmin,prcp,agdd_winter,tmax_winter,tmin_winter,prcp_winter
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1973,1973-02-06,37,26.78116,21.81633,6.745170,649.50,717.0,543.0,551
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1976,1976-01-28,28,26.51818,20.46606,4.892033,890.00,1031.0,749.0,631
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1977,1977-02-03,34,27.04046,21.68326,5.897222,875.00,1008.0,742.0,1265
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1978,1978-02-18,49,26.63037,21.04785,5.572299,927.00,1040.0,773.0,677
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1974,1974-01-20,20,26.76877,21.18828,6.367967,1003.50,1151.0,820.0,1390
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1981,1981-02-02,33,26.41593,21.22707,5.623529,829.00,934.0,682.0,652
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1982,1982-01-24,24,26.63745,20.84583,12.083688,733.95,555.0,435.0,1269
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1983,1983-02-09,40,27.22601,21.71941,10.823881,654.20,362.1,290.9,904
JAM00047918,Japan/Ishigakijima,24.33667,124.1644,5.7,1984,1984-01-26,26,26.72390,21.62684,11.536842,491.15,298.6,224.3,182


In [93]:
write.csv(result,"processed/clean.csv", row.names = FALSE)